-----
## BU NOTEBOOK İÇERİSİNDE TCP'nin ANA YAPISI ANLATILMAKTADIR. BİRİNCİ BÖLÜM AŞAĞIDA MEVCUTTUR.

---

# Python Socket Kütüphanesini Mantığıyla Öğrenmek

## 🧱 1. Socket Nedir?

Socket, iki cihaz (ya da iki uygulama) arasında **veri alışverişi** yapmak için kullanılan sanal bir “ağ uç noktasıdır”.

Basit düşün:  
- Telefon hattı var → bu **TCP bağlantısı**  
- Telefona konuşan iki kişi var → bunlar **client** ve **server**  
- Her birinin “kulaklık ve mikrofonu” var → bu da **socket**  

Bir cihaz “socket açarım, dinlerim” der, diğeri “socket açarım, bağlanırım” der.  
Böylece konuşma başlar.



## ⚙️ 2. Socket Türleri
Python’da 2 ana tür var:

- **SOCK_STREAM (TCP)** → güvenilir, sıralı, bağlantı odaklı (bizim işimiz bunla)
- **SOCK_DGRAM (UDP)** → bağlantısız, hızlı ama veri kaybına açık

Bugün hep `SOCK_STREAM` (TCP) üzerinden gideceğiz.


## 🌐 3. Socket’in Temel Akışı

### 🖥️ Sunucu tarafı
1. `socket()` — yeni bir ağ uç noktası oluşturur.  
2. `bind()` — hangi IP ve portta dinleyeceğini belirtir.  
3. `listen()` — “ben hazırım, biri bağlansın” der.  
4. `accept()` — biri bağlanınca kabul eder (ve yeni bir socket döner).  
5. `recv()` — istemciden veri alır.  
6. `send()` — istemciye veri gönderir.  
7. `close()` — bağlantıyı kapatır.

### 💻 İstemci tarafı
1. `socket()` — kendi ucunu oluşturur.  
2. `connect()` — sunucuya bağlanır.  
3. `send()` — mesaj gönderir.  
4. `recv()` — yanıtı alır.  
5. `close()` — bağlantıyı kapatır.

Bu kadar.  
Temelde “veri gönder–al” dediğin her şey bu zincirin üzerine oturur.


## 🔄 4. Socket Nasıl Çalışır (Basit Akış Diyagramı)



```bash 
[Client] [Server]
| socket() |
|---------------------------->| socket()
| | bind()
| | listen()
| connect() |
|---------------------------->| accept()
| send("selam!") |
|---------------------------->|
| | recv() → "selam!"
| | send("aleyküm selam")
| recv() → "aleyküm selam" |
|<----------------------------|
| close() | close()


İşte internetin özeti.


## 🧩 5. Mini Deney Planı

1. **Echo Server** → İstemciden ne gelirse onu geri yollar.  
2. **Basit Chat** → İki taraflı konuşma.  
3. **HTTP Denemesi** → Tarayıcıdan gelen istekleri anlamak.



----

# Bölüm A — TCP Echo (temel refleks)

* Amaç: Sunucuya ne gönderirsen, aynısını geri yollasın. Bu, TCP’nin “paket yapışması/yarılması” ve “kısmi okuma” gibi gerçeklerini görmenin en güvenli yolu

### 1) Tek-iş parçacıklı, bloklu Echo Sunucu

```python
import socket

HOST = "127.0.0.1"
PORT = 9000
BUF = 4096

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
srv.bind((HOST, PORT))
srv.listen(1)
print(f"[SERVER] listening on {HOST}:{PORT}")

#### 2️⃣ HOST = "127.0.0.1"

* Bu, sunucunun dinleyeceği IP adresi.

* "127.0.0.1" = loopback (localhost) adresi.
>Bu durumda sadece kendi bilgisayarından gelen istekleri kabul eder.

* Eğer "0.0.0.0" yazarsan, makinenin tüm ağ arayüzlerinden gelen istekleri kabul eder (yani dışarıdan erişilebilir hale gelir).

### 3️⃣ PORT = 9000

* Bu, sunucunun dinleyeceği kapı numarası (port).
>Her ağ uygulaması bir portta çalışır:

80 → HTTP

22 → SSH

443 → HTTPS

> Biz test için 9000 gibi yüksek numaralı (boşta olan) bir port seçtik.

### 4️⃣ BUF = 4096

* Bu, recv() ile okunacak veri tamponu boyutu.
* Yani istemciden veri alırken 4096 baytlık parça parça okuma yapacağız.
* TCP’de veri akışı “paket değil bayt akışı” olduğundan bu buffer boyutu okuma performansını etkiler.

### 5️⃣ with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:

* Bu satırda yeni bir socket nesnesi oluşturuluyor.

> AF_INET → IPv4 ailesi (örnek: 127.0.0.1).

> SOCK_STREAM → TCP protokolü (bağlantı odaklı, güvenilir).

* Yani: “IPv4 adresleriyle TCP bağlantıları kuracağım” demiş oluyorsun.

* with kullanmak, iş bittiğinde soketin otomatik kapanmasını sağlar (yani srv.close() gerekmez).

### 6️⃣ srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

* Bu, sistem düzeyinde bir ayardır:

> “Aynı adresi tekrar kullanmama izin ver.”

* Normalde bir TCP soketi kapandıktan sonra işletim sistemi o portu kısa süre “TIME_WAIT” durumunda tutar.
* Bu yüzden hemen yeniden başlatmaya çalışırsan “Address already in use” hatası alırsın.
* Bu ayar o sorunu engeller, geliştirme sırasında çok işe yarar.

### 7️⃣ srv.bind((HOST, PORT))

* Sunucu soketini belirttiğin IP adresine ve porta bağlar.
Bu şu anlama gelir:

> “İşletim sistemi, 127.0.0.1:9000 adresine gelen bağlantıları bu sokete yönlendirsin.”

* Artık sistem o portu senin uygulamana tahsis eder.
* Eğer port doluysa burada “OSError: [Errno 98] Address already in use” hatası alırsın.

## 🔁 Özetle Akış Şöyle:

* Socket oluşturulur.

* Aynı adresin yeniden kullanılmasına izin verilir.

* IP ve port’a bağlanılır (bind).

* Dinleme moduna alınır (listen).

> Artık accept() çağrısını bekleyebiliriz (yani biri bağlandığında onu kabul edeceğiz).

----

# 🧩 Bölüm B — TCP Echo Server (temel refleks)
### 💡 Amaç:

* Sunucuya ne gönderirsen, aynısını geri göndersin.
* Bu, TCP’nin veri akışını anlamak ve “sunucu-istemci” iletişiminin nasıl yürüdüğünü görmek için en yalın adımdır.

### ⚙️ 1️⃣ Akış Mantığı

* Sunucu başlar ve portu dinlemeye alır.
> “Burada biriyle konuşmak istiyorum.”

* İstemci bağlanır.
> Sunucu, accept() ile gelen bağlantıyı kabul eder.

* Sunucu gelen mesajı recv() ile okur.
> Mesaj genelde bytes türündedir; bu yüzden .decode() ile yazıya çeviririz.

* Sunucu aldığı mesajı send() ile geri yollar.
> Aynı mesajı gönderir — tam bir yankı (echo) efekti.

* Bağlantı kapanır veya yeni istemci beklenir.

### 🧠 Önemli Noktalar
#### 🔹 accept()

* Bu fonksiyon blokludur (yani biri bağlanana kadar kod durur).

Döndürdüğü iki değer vardır:

>client_socket → istemciyle haberleşeceğin özel kanal

>client_addr → istemcinin adres bilgisi (IP, port)

* Her yeni bağlantı için işletim sistemi yeni bir socket oluşturur.
* Yani srv dinleme görevinde kalır, client_socket ise konuşma kanalını yürütür.

### 🔹 recv(BUF)

* İstemciden gelen veriyi okur.
* Eğer istemci çok büyük veri gönderirse, bu işlemi döngüyle sürdürmek gerekebilir.
* Ama ilk aşamada tek okuma yeterli.

### 🔹 send()

* Sunucu, veriyi bytes olarak göndermeli.
* Bu yüzden string veriyi .encode() ile byte’a dönüştürürüz.

### 🔹 close()

* Her bağlantı bittiğinde client_socket.close() çağrılır.
* Böylece bağlantı sonlandırılır ve yeni bağlantılar kabul edilebilir.

### 🔁 Özetle Akış:

```bash
[Server]                                 [Client]
socket()                                socket()
bind()                                  connect()
listen()         <-------------------   |
accept()         ------------------->   |
recv()           <-------------------   send("selam")
send("selam")    ------------------->   recv("selam")
close()                                 close()


## 💻 Kod — Tek İş Parçacıklı Echo Server

In [ ]:
import socket

HOST = "127.0.0.1"
PORT = 8082
BUF = 4096

# Sunucuyu başlat
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"[SERVER] listening on {HOST}:{PORT}")

    while True:
        # 1️⃣ Yeni bağlantı kabul et
        client_socket, client_addr = srv.accept()
        print(f"[+] Connected by {client_addr}")

        # 2️⃣ İstek (mesaj) al
        data = client_socket.recv(BUF)
        if not data:
            print("[-] Empty data, closing connection")
            client_socket.close()
            continue

        # 3️⃣ Veriyi ekrana yaz
        msg = data.decode("utf-8")
        print(f"[CLIENT]: {msg}")

        # 4️⃣ Aynı mesajı geri gönder (echo)
        client_socket.sendall(data)

        # 5️⃣ Bağlantıyı kapat
        client_socket.close()
        print(f"[x] Connection closed: {client_addr}\n")

[SERVER] listening on 127.0.0.1:8082


---
# 🧩 Aşama C — TCP Echo İstemci (Client)
### 💡 Amaç:

* Sunucuya bağlan, mesaj gönder, aynı mesajı geri al (yani yankıyı duy).

### 🧠 Mantık

İstemci tarafında işler çok daha kısa:

* socket() → kendi uç noktasını oluşturur

* connect((HOST, PORT)) → sunucuya bağlanır

* sendall() → mesajı yollar

* recv() → sunucudan dönen yanıtı alır

* close() → bağlantıyı kapatır

## 💻 Kod — Echo Client

In [ ]:
import socket

HOST = "127.0.0.1"  # Sunucu adresi
PORT = 8082         # Sunucu portu (sunucudakiyle aynı olmalı)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client:
    client.connect((HOST, PORT))
    print("[CLIENT] Connected to server")

    # Mesajı gönder
    msg = "selam gpt, echo testi!"
    client.sendall(msg.encode("utf-8"))

    # Yanıtı al
    data = client.recv(4096)
    print(f"[SERVER]: {data.decode('utf-8')}")


### 🧩 Nasıl test edeceksin

* Sunucuyu açık bırak.
> Yani "[SERVER] listening..." yazısı ekranda kalmalı.

* Yeni bir terminal aç → bu istemci kodunu çalıştır:

**python echo_client.py**


* Sonuç olarak iki terminalde de şunu görmelisin:

```bash
[SERVER] listening on 127.0.0.1:8082
[+] Connected by ('127.0.0.1', 56324)
[CLIENT]: selam gpt, echo testi!
[x] Connection closed: ('127.0.0.1', 56324)

-------

# 🧠 TCP Veri Kanalının Mantığı

Bir TCP bağlantısı kurulduğunda sistem şu sırayla işler:

* Sunucu: listen() → “bağlantıya hazırım”

* İstemci: connect() → “bağlanmak istiyorum”

* Sunucu: accept() → “tamam, bağlantı kuruldu”

* İstemci: sendall() → byte akışı gönderir

* Sunucu: recv() → byte akışını alır

* Sunucu: sendall() → aynı veriyi geri yollar (bizim echo kısmı)

* İstemci: recv() → cevabı alır

* Her iki taraf da: close() → bağlantıyı kapatır

>TCP bu noktada “güvenilir aktarım” sağlar; her byte karşıya ulaştığında ACK sinyali döner.

>Yani TCP, senin “veri” kanalında hem sıralamayı hem teslimatı garanti eder.

>Bu yüzden HTTP, WebSocket, SSH, SMTP gibi bütün protokoller TCP üzerine kurulur.

## 🔍 Veri Kanalını Gözlemleme (Detaylı Kod)

* Aşağıda hem sunucu hem istemci tarafında gelen–giden byte’ları ekrana yazdıran sürüm var.

#### **Bu, TCP akışının çıplak halini görmeni sağlar.**

###  echo_server

In [ ]:
import socket

HOST = "127.0.0.1"
PORT = 8082
BUF = 4096

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"[SERVER] listening on {HOST}:{PORT}")

    while True:
        client_socket, client_addr = srv.accept()
        print(f"[+] Connected by {client_addr}")

        # Veri alma
        data = client_socket.recv(BUF)
        if not data:
            client_socket.close()
            continue

        print(f"[SERVER:RECV_RAW] {data}")
        print(f"[SERVER:RECV_DECODED] {data.decode('utf-8')}")

        # Veri gönderme
        client_socket.sendall(data)
        print(f"[SERVER:SENT] {data}")

        client_socket.close()
        print(f"[x] Connection closed: {client_addr}\n")

### echo_client

In [ ]:
import socket
import time

HOST = "127.0.0.1"
PORT = 8082
BUF = 4096

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client:
    client.connect((HOST, PORT))
    print("[CLIENT] Connected to server")

    msg = "selam gpt, veri kanal testi 🔁"
    data = msg.encode("utf-8")

    print(f"[CLIENT:SEND_RAW] {data}")
    client.sendall(data)
    print(f"[CLIENT:SENT] {msg}")

    # Sunucudan yanıtı al
    response = client.recv(BUF)
    print(f"[CLIENT:RECV_RAW] {response}")
    print(f"[CLIENT:RECV_DECODED] {response.decode('utf-8')}")

    client.close()


## 📡 Çıktı Örneği

* Sunucu terminali:
 ```bash
[SERVER] listening on 127.0.0.1:8082
[+] Connected by ('127.0.0.1', 56142)
[SERVER:RECV_RAW] b'selam gpt, veri kanal testi \xf0\x9f\x94\x81'
[SERVER:RECV_DECODED] selam gpt, veri kanal testi 🔁
[SERVER:SENT] b'selam gpt, veri kanal testi \xf0\x9f\x94\x81'
[x] Connection closed: ('127.0.0.1', 56142)
```


* İstemci terminali:
```bash
[CLIENT] Connected to server
[CLIENT:SEND_RAW] b'selam gpt, veri kanal testi \xf0\x9f\x94\x81'
[CLIENT:SENT] selam gpt, veri kanal testi 🔁
[CLIENT:RECV_RAW] b'selam gpt, veri kanal testi \xf0\x9f\x94\x81'
[CLIENT:RECV_DECODED] selam gpt, veri kanal testi 🔁

#### Bu görüntü aslında TCP’nin “byte akışı”nın anatomisi:

> b'...' → ham byte dizisi

> .decode('utf-8') → insan tarafından okunabilir hali

* İşte bu kanal, HTTP’nin de veri taşıyıcısı olacak.

##### Bir sonraki adımda bu “byte akışı”nın içine HTTP başlıkları ve gövdesini yerleştirip, tarayıcıyı konuşturacağız.